In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
from torchvision.utils import save_image
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import numpy as np
import pandas as pd
import os

In [3]:
dataroot = '../input/plant-seedlings-classification/train'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
img_shape = 128
latent_dim = 100
batch_size = 16
sample_interval = 400

In [5]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(img_shape),
                               transforms.CenterCrop(img_shape),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True)

In [6]:
dataset.class_to_idx

In [7]:
# Plot some training images
real = next(iter(dataloader))
plt.figure(figsize=(10,10))
plt.title("Example Images")
plt.imshow(np.transpose(vutils.make_grid(real[0].to(device), padding=2, normalize=True).cpu(),(1,2,0)))

In [8]:
print(real[0].shape)

In [9]:
n_classes = 12
embedding_dim = 100

In [10]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [11]:
def create_noise(samples):
    return torch.randn(samples, latent_dim+10, device=device)
final_noise = create_noise(64).to(device)

In [12]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
     
        self.label_conditioned_generator = nn.Sequential(nn.Embedding(n_classes, embedding_dim),
                      nn.Linear(embedding_dim, 16))
        
    
        self.latent = nn.Sequential(nn.Linear(latent_dim, 4*4*512),
                                   nn.LeakyReLU(0.2, inplace=True))
           

        self.model = nn.Sequential(nn.ConvTranspose2d(513, 64*8, 4, 2, 1, bias=False),
                      nn.BatchNorm2d(64*8, momentum=0.1,  eps=0.8),
                      nn.ReLU(True),

                      nn.ConvTranspose2d(64*8, 64*4, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*4, momentum=0.1,  eps=0.8),
                      nn.ReLU(True), 
                                   
                      nn.ConvTranspose2d(64*4, 64*2, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*2, momentum=0.1,  eps=0.8),
                      nn.ReLU(True),
                                   
                      nn.ConvTranspose2d(64*2, 64*1, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*1, momentum=0.1,  eps=0.8),
                      nn.ReLU(True),
                                   
                      nn.ConvTranspose2d(64*1, 3, 4, 2, 1, bias=False),
                      nn.Tanh())

    def forward(self, inputs):
        noise_vector, label = inputs
        label_output = self.label_conditioned_generator(label)
        label_output = label_output.view(-1, 1, 4, 4)
        latent_output = self.latent(noise_vector)
        latent_output = latent_output.view(-1, 512,4,4)
        concat = torch.cat((latent_output, label_output), dim=1)
#         print(concat.shape)
        image = self.model(concat)
        #print(image.size())
        return image

In [13]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
    
        self.label_condition_disc = nn.Sequential(nn.Embedding(n_classes, embedding_dim),
                      nn.Linear(embedding_dim, 3*img_shape*img_shape))
             
        self.model = nn.Sequential(nn.Conv2d(6, 64, 4, 2, 1, bias=False),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64, 64*2, 4, 3, 2, bias=False),
                      nn.BatchNorm2d(64*2, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64*2, 64*4, 4, 3,2, bias=False),
                      nn.BatchNorm2d(64*4, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64*4, 64*8, 4, 3, 2, bias=False),
                      nn.BatchNorm2d(64*8, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True), 
                      nn.Flatten(),
                      nn.Dropout(0.4),
                      nn.Linear(4608, 1),
                      nn.Sigmoid()
                     )

    def forward(self, inputs):
        img, label = inputs
        label_output = self.label_condition_disc(label)
        label_output = label_output.view(-1, 3, img_shape, img_shape)
#         print(img.shape)
#         print(label_output.shape)
#         print("done")
        concat = torch.cat((img, label_output), dim=1)
        #print(concat.size())
        output = self.model(concat)
        return output

In [ ]:
generator = Generator().to(device)
generator.apply(weights_init)

discriminator = Discriminator().to(device)
discriminator.apply(weights_init)

In [ ]:
criterion = nn.BCELoss(reduction='mean')
D_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
G_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

def generator_loss(label, fake_output):
    gen_loss = binary_cross_entropy(label, fake_output)
    #print(gen_loss)
    return gen_loss

def discriminator_loss(label, output):
    disc_loss = binary_cross_entropy(label, output)
    #print(total_loss)
    return disc_loss

In [ ]:
final_noise = torch.randn(batch_size, latent_dim, device=device)  
final_noise = final_noise.to(device)

In [ ]:
# anim = []
# num_epochs = 100
# for epoch in range(num_epochs): 

#     D_loss_list, G_loss_list = [], []

    
#     for index, (real_images, labels) in enumerate(dataloader):
#         D_optimizer.zero_grad()
#         real_images = real_images.to(device)
#         labels = labels.to(device)
#         labels = labels.unsqueeze(1).long()

      
#         real_target = Variable(torch.ones(real_images.size(0), 1).to(device))
#         fake_target = Variable(torch.zeros(real_images.size(0), 1).to(device))
      
#         D_real_loss = criterion(discriminator((real_images, labels)), real_target)
#         # print(discriminator(real_images))
#         #D_real_loss.backward()
    
#         noise_vector = torch.randn(real_images.size(0), latent_dim, device=device)  
#         noise_vector = noise_vector.to(device)
        
       
#         generated_image = generator((noise_vector, labels))
#         output = discriminator((generated_image.detach(), labels))
#         D_fake_loss = criterion(output,  fake_target)

    
#         # train with fake
#         #D_fake_loss.backward()
      
#         D_total_loss = (D_real_loss + D_fake_loss) / 2
#         D_loss_list.append(D_total_loss)
      
#         D_total_loss.backward()
#         D_optimizer.step()

#         # Train generator with real labels
#         G_optimizer.zero_grad()
#         G_loss = criterion(discriminator((generated_image, labels)), real_target)
#         G_loss_list.append(G_loss)

#         G_loss.backward()
#         G_optimizer.step()
        
#         if index%100 == 0: 
#             final_images = generator((final_noise, labels)).detach()
#             final_images = vutils.make_grid(final_images.to(device), padding=2, normalize=True)
#             anim.append(final_images)
        
#     print(f"Epoch: {epoch+1} , Gen Loss: {G_loss} , Disc Loss: { D_total_loss}")

In [ ]:
# # save the generator model
# from datetime import datetime
# gen_filename = datetime.now().strftime('/kaggle/working/%d-%m-%y-%H_%M_cgan_genweights.h5')
# disc_filename = datetime.now().strftime('/kaggle/working/%d-%m-%y-%H_%M_cgan_discweights.h5')
# torch.save(generator, gen_filename)
# torch.save(discriminator, disc_filename)

In [40]:
gen256_filename = '../input/outputfile/02-03-22-18_34_cgan_genweights.h5'

gen256 = torch.load(gen_256_filename, map_location=torch.device('cpu'))
gen256.eval()

In [16]:
gen128_filename = '../input/outputfile/03-03-22-06_12_cgan_genweights.h5'

gen128 = torch.load(gen128_filename, map_location=torch.device('cpu'))
gen128.eval()

In [29]:
seed_type = os.listdir("/kaggle/input/plant-seedlings-classification/train")

In [60]:
label = 5
batch_size = 5
labels = torch.ones(batch_size,device=device) * label
print(labels)
labels = labels.to(device)
labels = labels.unsqueeze(1).long()
noise = torch.randn(batch_size, latent_dim, device=device).to(device)
print(noise.shape)
predictions = gen256((noise, labels))
images = predictions.detach()
images = images.to(device)
images = vutils.make_grid(images, padding=2, normalize=True)
plt.imshow(np.transpose(images.cpu(),(1,2,0)))

In [62]:
batch_size = 5
label = 5
labels = torch.ones(batch_size,device=device) * label
labels = labels.to(device)
labels = labels.unsqueeze(1).long()
noise = torch.randn(batch_size, latent_dim, device=device).to(device)
print(labels,seed_type[label])
predictions = gen128((noise, labels))
images = predictions.detach()
images = images.to(device)
images = vutils.make_grid(images, padding=2, normalize=True)
plt.imshow(np.transpose(images.cpu(),(1,2,0)))

In [55]:
def generate_images(model, num_img, n_classes=10):
 
  # This is so all layers run in inference mode (batchnorm).
    for label in range(n_classes):
        root = '/kaggle/working/output_images/'+ str(label)
        os.mkdir(root) if not os.path.exists(root) else None
        
        labels = torch.ones(1,device=device)* label
        labels = labels.to(device)
        labels = labels.unsqueeze(1).long()
        for i in range(num_img):
            noise = torch.randn(1, latent_dim, device=device).to(device)
            predictions = gen((noise, labels))
            images = predictions.detach()
            images = vutils.make_grid(images.to(device), padding=2, normalize=True)
            
            save_image(images, os.path.join(root, f'image_{i}.png'))


In [ ]:
from torchvision.utils import save_image
num_examples_to_generate = 10
latent_dim = 100
n_classes = 12
generate_images(gen, num_examples_to_generate,  n_classes)

In [ ]:
root = '/kaggle/working/output_images/0/'
images = os.listdir(root)

In [ ]:
images = [os.path.join(root, img) for img in images]